# Advanced Querying Mongo

Importing libraries and setting up connection

In [4]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [5]:
db = client.get_database("ironhack")

In [6]:
db.list_collection_names()

['restaurants', 'companies', 'resaurant']

In [7]:
c=db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [8]:
Babelgum = list(c.find({"name": "Babelgum"}))


In [10]:
#print(Babelgum)

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [11]:
more_5000 = {"number_of_employees":{"$gt": 5000}}


In [16]:
n_employees = list(c.find(more_5000).limit(20))


In [18]:
#print(n_employees)

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [29]:
proy = {"name":1, "founded_year":1, "_id":0}

cond1 = {"founded_year":{"$gte": 2000}}
cond2 = {"founded_year":{"$lte": 2005}}

companies = list(c.find({"$and": [cond1,cond2]},proy))


In [32]:
#print(companies)

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [39]:
proy = {"name":1, "ipo":1, "_id":0}

cond1 = {"ipo.valuation_amount":{"$gt": 100000000}}
cond2 = {"founded_year":{"$lt": 2010}}

companies2 = list(c.find({"$and": [cond1,cond2]},proy))

In [41]:
#companies2

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [58]:
cond1 = {"number_of_employees":{"$lt": 1000}}
cond2 = {"founded_year":{"$lt": 2005}}

companies3 = list(c.find({"$and": [cond1,cond2]}).sort("number_of_employees",1).limit(10))

# No estoy segura si tengo que añadir "1" al sort o si al ordenar por nº de empleados, lo ordena por defecto de forma ascendente.

In [60]:
#companies3

### 6. All the companies that don't include the `partners` field.

In [66]:
#list(c.find({"partners": {"$nin": [2,3,4]}})

In [74]:
companies4 = list(c.find({"partners": {"$exists": False}}))

In [68]:
#companies4

In [71]:
len(companies4) # The len when exists:True is the same as the lenght of companies. The are not any False values.

18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [83]:
companies5 = list(c.find({"category_code": {"$type": "null"}}))

In [85]:
#companies5

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [87]:
cond1 = {"number_of_employees":{"$gte": 100}}
cond2 = {"number_of_employees":{"$lte": 1000}}
proy = {"name":1, "number":1, "_id":0}

companies6 = list(c.find({"$and": [cond1,cond2]}))

In [89]:
#companies6

### 9. Order all the companies by their IPO price in a descending order.

In [95]:
companies7 = list(c.find({}).sort("ipo",-1))

In [97]:
#companies7

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [99]:
companies8 = list(c.find({}).sort("number_of_employees",-1).limit(10))

In [101]:
#companies8

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [102]:
cond1={"founded_month":{"$gt":6}}

companies9 = list(c.find(cond1).limit(1000))

In [104]:
#companies9

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [105]:
cond1={"founded_year":{"$lt":2000}}
cond2={"acquisition.price_amount":{"$gt":1000000}}

companies10 = list(c.find({"$and": [cond1,cond2]}))


In [106]:
#companies10

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [111]:
cond1={"acquisition.acquired_year":{"$gt":2010}}
proy = {"name":1, "acquisition":1, "_id":0}

companies11 = list(c.find(cond1,proy).sort("acquisition.price_amount",-1))


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [116]:
proy = {"founded year":1, "name":1, "_id":0}

companies12 = list(c.find({},proy).sort("founded_year",1))

In [118]:
#companies12

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [122]:
cond1={"founded_day":{"$lte":7}}
companies13 = list(c.find(cond1).sort("acquisition.price_amount",-1).limit(10))

In [123]:
#companies13

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [127]:
cond1={"number_of_employees":{"$gt":4000}}
cond2={"category_code": "web"}

companies14 = list(c.find({"$and": [cond1,cond2]}).sort("number_of_employees",1))


In [129]:
#companies14

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [132]:
cond1={"acquisition.price_amount":{"$gt":10000000}}
cond2={"acquisition.price_currency_code": "EUR"}

companies15 = list(c.find({"$and": [cond1,cond2]}))

In [134]:
#companies15

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [140]:
cond1={"acquisition.acquired_month":{"$lt":4}}
proy = {"name":1, "acquisition":1, "_id":0}

companies16 = list(c.find(cond1,proy).limit(10))



In [142]:
#companies16

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [147]:
proyect = {"name": 1, 'founded_year': 1, 'acquisition.acquired_year':1, '_id': 0}
cond1 = {'founded_year': {"$in": list(range(2000,2011))}}
cond2 = {'acquisition.acquired_year': {"$nin": list(range(0,2012))}}
companies19 = list(c.find({"$and": [cond1,cond2]}, proyect).limit(3))

In [149]:
#companies19

### 20. All the companies that have been 'deadpooled' after the third year.